In [1]:
%% morning matlab %%

## Convert from BCI2000

In [2]:
%% BCI2000 to matlab conversion

addpath(genpath('/home/brooke/knight_lab_commons/loading_data_scripts/BCI2000mex'))
path_data = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH016/PacmanTask/ECOGS001R01.dat'
[signal,states,parameters] = load_bcidat(path_data);


path_data =

    '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH016/PacmanTask/ECOGS001R01.dat'



In [ ]:
plot(states.Trial_on_off(1:1000000))

In [16]:
%% Now add other paths %%
% add fieldtrip
ftDir = '/home/brooke/knight_server/remote/lbellier/DataWorkspace/_tools/git/fieldtrip/';
if exist('ft_defaults.m', 'file') == 0
    addpath(ftDir); ft_defaults;
end
% Ludo's tools
addpath('/home/brooke/knight_server/remote/lbellier/DataWorkspace/_tools/');

%% Add 'photodiode'
signal_photo = [signal, states.Trial_on_off];
parameters.ChannelNames.Value{273} =  'photodiode';

%% Convert to fieldtrip
data = ft_array2data(double(signal_photo),  parameters.SamplingRate.NumericValue, parameters.ChannelNames.Value);

%% save %%
save('/home/brooke/knight_server/remote/bstavel/pacman_ieeg_data/BJH016/BJH016_raw_data.mat', 'data', '-v7.3');

## After conversion from BCI2000

begin here after data from BCI2000 has been converted to fieldtrip

In [11]:
%% add field trip %%
ftDir = '/home/brooke/fieldtrip/fieldtrip-20191213';
if exist('ft_defaults.m', 'file') == 0
    addpath(ftDir); ft_defaults;
end

addpath(genpath('/usr/local/MATLAB/R2021b/toolbox/signal/'))

In [ ]:
%% load data %%

% paths %
base_dir = '/home/brooke/knight_server/remote/bstavel/'
bj16_path = '/home/brooke/knight_server/remote/bstavel/pacman_ieeg_data/BJH016/BJH016_raw_data.mat'

%% loda data %%
load(bj16_path)

In [15]:
data


data = 

  struct with fields:

       fsample: 2000
    sampleinfo: [1 5901900]
         trial: {[273x5901900 uint8]}
          time: {[0 5.0000e-04 1.0000e-03 0.0015 0.0020 0.0025 0.0030 ... ]}
         label: {273x1 cell}



In [17]:
%% Downsample %%
cfg = [];
cfg.resamplefs = 1080;
data = ft_resampledata(cfg, data);

the input is raw data with 273 channels and 1 trials
the call to "ft_selectdata" took 0 seconds and required the additional allocation of an estimated 1 MB
resampling data


Unrecognized function or variable 'eml_bitand'.

Error in kaiser (line 41)
iseven = ONE - eml_bitand(nw,ONE);

Error in signal.internal.resample.createFilter (line 28)
        h = firls( L-1, [0 2*fc 2*fc 1], [1 1 0 0]).*kaiser(L,bta)';

Error in signal.internal.resample.uniformResample (line 43)
[h,hout,delay] = signal.internal.resample.createFilter(x,Lx,p,q,N,bta);

Error in resample (line 198)
            signal.internal.resample.uniformResample(xIn, isDimValSet, Dim, dimIn, numericArgs{:});

Error in ft_resampledata (line 231)
        newdat = transpose(resample(transpose(olddat),fsres,fsorig));



In [14]:
%% Detrend and notch clinical data
HPcutoff = 1; % high-pass filter cutoff
LPcutoff = 200; % low-pass filter cutoff change to 150
powerLineF0 = 60;
cfg = [];
cfg.bsfilter = 'yes';
cfg.bsfreq = (powerLineF0 * (1:round((LPcutoff+powerLineF0)/powerLineF0)) + [-1; 1])';
if data.fsample <= 512
    cfg.bsfreq = cfg.bsfreq(1:end-1, :); % for fs=512Hz, e.g. OS13, OS21
end
cfg.bsfiltord = 3;
cfg.hpfilter = 'yes';
cfg.hpfreq = HPcutoff;
cfg.hpfiltord = 3;
cfg.lpfilter = 'yes';
cfg.lpfreq = LPcutoff;
cfg.lpfiltord = 3;
filtered_data = ft_preprocessing(cfg, data);


the call to "ft_selectdata" took 0 seconds and required the additional allocation of an estimated 0 MB
preprocessing


Unable to resolve the name signalwavelet.internal.isodd.

Error in isodd (line 9)
tf = signalwavelet.internal.isodd(x);

Error in buttap (line 44)
if isodd(n) % n is odd

Error in butter>butterImpl (line 101)
[zs,ps,ks] = buttap(n1);

Error in butter (line 59)
   [varargout{1:nargout}] = butterImpl(n,Wn,varargin{:});

Error in ft_preproc_lowpassfilter (line 156)
    [B, A] = butter(N, max(Flp)/Fn);

Error in preproc (line 388)
if strcmp(cfg.lpfilter, 'yes'),     dat = ft_preproc_lowpassfilter(dat, fsample, cfg.lpfreq, cfg.lpfiltord, cfg.lpfilttype, cfg.lpfiltdir, cfg.lpinstabilityfix, cfg.lpfiltdf, cfg.lpfiltwintype, cfg.lpfiltdev, cfg.plotfiltresp, cfg.usefftfilt); end

Error in ft_preprocessing (line 377)
    [dataout.trial{i}, dataout.label, dataout.time{i}, cfg] = preproc(data.trial{i}, data.label,  data.time{i}, cfg, begpadding, endpadding);



In [ ]:
%plot native
%% display %%
cfg = [];
cfg.viewmode = 'vertical';
cfg.preproc.demean = 'yes';
% cfg.blocksize = 10;
% cfg.channel = goodChansDispIdx;
% cfg.position = pos2;
% cfg.ylim = [-1 1] * .0005;
ft_databrowser(cfg, data);

In [ ]:
%% display %%
cfg = [];
cfg.viewmode = 'vertical';
cfg.preproc.demean = 'yes';
% cfg.blocksize = 10;
% cfg.channel = goodChansDispIdx;
% cfg.position = pos2;
% cfg.ylim = [-1 1] * .0005;
ft_databrowser(cfg, data);

In [ ]:
data

# Cleaning Notes with Bob
## 1/19/2022

Clean data, but with a lot of spiking activity

Epileptic channels: Gl1, GL2, GL3, HL1, HL2

Noisy channels: BL1, F8, EL1, EL2 , HL14, HL15, HL16

Epochs that may need to be removed (depending on how the data looks after removing epileptic channels + nearby channels): LOOK AT THIS AGAIN FOR THIS PATEINT
`116-117, 127.5:129, 160-161.5, 343-345, 540-541, 6216-623, 681-683`
